# Fine-tuning LLM

## Install and import libraries

In [3]:
!pip install transformers
!pip install datasets
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 3.8 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import torch

from datasets import load_dataset
from datasets import load_metric

from transformers import AutoConfig
from transformers import AutoModelForCausalLM # Zero-shot LLaMA-2-7B
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import pipeline
from transformers import TrainingArguments
from transformers import Trainer

# Modify and push files
from huggingface_hub import login, logout
from huggingface_hub import HfApi

## Data import from HuggingFace

Change code below to take from HungingFace instead of gtfintechlab

In [5]:
HUGGING_FACE_LINK = "roymgabriel/BioPharma"

In [6]:
data_files = {"train": "train.csv", "test": "test.csv"}
dataset = load_dataset(HUGGING_FACE_LINK, data_files=data_files)
print(dataset)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['ticker', 'disease', 'stage', 'date', 'catalyst', 'label'],
        num_rows: 6450
    })
    test: Dataset({
        features: ['ticker', 'disease', 'stage', 'date', 'catalyst', 'label'],
        num_rows: 785
    })
})


## Fine-Tune TinySapBERT + TinyPubMedBERT model
Taken from: dmis-lab/TinySapBERT-from-TinyPubMedBERT-v1.0

[TinySapBERT](https://aclanthology.org/2021.naacl-main.334.pdf) "TinySapBERT", tiny-sized biomedical entity representations (language model) trained using official SapBERT code and instructions (Liu et al., NAACL 2021). It is used in conjuction with TinyPubMedBERT for optimal classification.

### Data processing and tokenization

The code below uses the AutoTokenizer from `'roberta-base'`. This is important for several reasons:

* <b>Model-Specific Tokenization</b>: Different models can have different tokenization approaches. For example:

  * RoBERTa uses Byte-Pair Encoding (BPE).
  * [BERT](https://arxiv.org/abs/1810.04805) uses WordPiece tokenization.

* <b>Vocabulary Matching</b>: Each pretrained model comes with a specific vocabulary that it has been trained on.

* <b>Model Configuration and Special Tokens</b>: Pretrained models often come with specific configurations, including special tokens (like padding tokens, mask tokens, etc.).

* <b>Preprocessing Consistency</b>: If you are fine-tuning a pretrained model on a new task or dataset, it's important to preprocess the new data like how the original training data was processed.

In [7]:
# allenai/scibert_scivocab_uncased
# microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext
# dmis-lab/biobert-v1.1
# bert-base-uncased
# dmis-lab/TinySapBERT-from-TinyPubMedBERT-v1.0

MODEL = 'microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext'
tokenizer = AutoTokenizer.from_pretrained(MODEL)

def tokenize_data(example):
    return tokenizer(example['catalyst'], padding="max_length", truncation=True, max_length=512)

dataset_new = dataset.map(tokenize_data, batched=True)

remove_columns = ['ticker', 'disease', 'stage', 'date', 'catalyst']
dataset_new = dataset_new.map(remove_columns=remove_columns)


print(dataset_new)

train_dataset = dataset_new['train']
eval_dataset = dataset_new['test']

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

Map:   0%|          | 0/6450 [00:00<?, ? examples/s]

Map:   0%|          | 0/785 [00:00<?, ? examples/s]

Map:   0%|          | 0/6450 [00:00<?, ? examples/s]

Map:   0%|          | 0/785 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6450
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 785
    })
})


### Set training arguments

`learning_rate`: Anywhere from 1e-5 to 1e-7 typically works well.

`push_to_hub`: False for now because we don't want to push to Hugging Face until we are happy with the model.

In [8]:
training_args = TrainingArguments(output_dir=HUGGING_FACE_LINK,
                                  num_train_epochs=100,
                                  learning_rate=1e-6,
                                  per_device_train_batch_size=4,
                                  hub_model_id=HUGGING_FACE_LINK,
                                  push_to_hub=False)

### Load Pre-trained Language Model (PLM)

The code below loads the pretrained model "roberta-base" from Hugging Face's models hub. Note that sequence classification is a task where a model assigns a label to an entire sequence (or sentence) rather than individual tokens.

In [9]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=3)

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Load and create function to compute metric

This F1 score is weighted by the number of true instances for each label. It accounts for class imbalance by giving more weight to the classes with more instances.

In [10]:
metric = load_metric("f1")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")

<ipython-input-10-837098ea659f>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("f1")


### Create trainer object

The shard method is used to divide the dataset into multiple smaller "shards" and then select one of those shards. In this case, both the training and evaluation datasets are divided into 10 shards, and only the first shard (index=0) is selected for training and evaluation. This is useful when you want to train or evaluate on a subset of the data, possibly for faster experimentation.

In [11]:
import gc
gc.collect()

48

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset.shard(num_shards=10, index=0),
    eval_dataset=eval_dataset.shard(num_shards=10, index=0),
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

### Train (Fine-tune) the model

In [13]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,1.033400
1000,0.908300
1500,0.831300
2000,0.783700
2500,0.741400
3000,0.709500
3500,0.665100
4000,0.646700
4500,0.612900
5000,0.609800


TrainOutput(global_step=16200, training_loss=0.5647083640392916, metrics={'train_runtime': 6791.7626, 'train_samples_per_second': 9.497, 'train_steps_per_second': 2.385, 'total_flos': 1.6970815443456e+16, 'train_loss': 0.5647083640392916, 'epoch': 100.0})

### Evaluate the model

In [14]:
evaluate_output = trainer.evaluate()
print(evaluate_output)
# PUBMEDBERT without SapBERT

{'eval_loss': 1.4248623847961426, 'eval_f1': 0.7014281957165562, 'eval_runtime': 2.7305, 'eval_samples_per_second': 28.933, 'eval_steps_per_second': 3.662, 'epoch': 100.0}


## Deploy model on HuggingFace

### Login to HuggingFace

This function call prompts the user to log in to their Hugging Face account. Once authenticated, an access token will be saved locally, allowing the user to interact with the Hugging Face Hub (e.g., push models, datasets) programmatically without needing to re-authenticate every time.

In [ ]:
login()

### Push tokenizer and trained model

After pushing, you can check out the model on Hugging Face. It is also possible to do some inference on Hugging Face (test the model).

In [ ]:
MODEL_PATH = "roymgabriel/BioPharma"
tokenizer.push_to_hub("roymgabriel/BioPharma")
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/57.4M [00:00<?, ?B/s]

events.out.tfevents.1701240478.deaaef5c326c.201.4:   0%|          | 0.00/4.50k [00:00<?, ?B/s]

events.out.tfevents.1701240507.deaaef5c326c.201.5:   0%|          | 0.00/4.50k [00:00<?, ?B/s]

Upload 6 LFS files:   0%|          | 0/6 [00:00<?, ?it/s]

events.out.tfevents.1701240523.deaaef5c326c.201.6:   0%|          | 0.00/6.42k [00:00<?, ?B/s]

events.out.tfevents.1701244181.deaaef5c326c.201.7:   0%|          | 0.00/405 [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

'https://huggingface.co/roymgabriel/BioPharma/tree/main/'

### Modify and push additional files

The code below modifies a local tokenizer configuration file, then uploads the updated configuration to a specified repository on the Hugging Face Model Hub.

In [ ]:
with open("/content/roymgabriel/BioPharma/tokenizer_config.json", "r") as f:
  config = json.load(f)

# Make the necessary changes to the config file.

config["name_or_path"] = MODEL

with open("/content/roymgabriel/BioPharma/tokenizer_config.json", "w") as f:
  json.dump(config, f, indent=4)

api = HfApi()
api.upload_file(
    path_or_fileobj="/content/roymgabriel/BioPharma/tokenizer_config.json",
    path_in_repo="tokenizer_config.json",
    repo_id="roymgabriel/BioPharma",
    repo_type="model",
)

'https://huggingface.co/roymgabriel/BioPharma/blob/main/tokenizer_config.json'

## Logout from HuggingFace
Logout so your tokens won’t be used by someone else.

In [ ]:
logout() # logout completely

Successfully logged out.
